In [3]:

import sys
sys.path.append(r"E:\labs\ML Lab\ML Lab Spring 25\timeseires")

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model

import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [6]:
def MLP():
    model = Sequential()
    model.add(Flatten(input_shape=(time_steps , num_features)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))
    return model

In [7]:
model1 = MLP()
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 504)               0         
_________________________________________________________________
dense (Dense)                (None, 32)                16160     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 16,193
Trainable params: 16,193
Non-trainable params: 0
_________________________________________________________________


In [8]:
tensorflow.keras.utils.plot_model(model1 )

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [10]:
checkpoints = r'E:\labs\ML Lab\lab7\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'E:\labs\ML Lab\lab7'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [11]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
#TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1]

In [12]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =MLP()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [14]:
import os
path_dataset =r'E:\Asad python\dada sets\AEP\AEP'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [15]:
df_te.head(1)

,aep,Is_holiday1,Is_holiday2,Is_Weekend1,Is_Weekend2,sin_month,cos_month,sin_week,cos_week,sin_hour,...,sin_wintert,cos_wintert,sin_springt,cos_springt,sin_summert,cos_summert,sin_fallt,cos_fallt,sin_year_dayt,cos_year_dayt
0,0.518532,1,0,1,0,1.0,6.120000e-17,0.866025,-0.5,-0.707107,...,0,1.0,1,6.120000e-17,0,1.0,0,1.0,0.956235,0.2926


In [16]:
test_set[0]

array([ 5.18532385e-01,  1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
        0.00000000e+00,  1.00000000e+00,  6.12000000e-17,  8.66025404e-01,
       -5.00000000e-01, -7.07106781e-01, -7.07106781e-01,  0.00000000e+00,
        1.00000000e+00,  1.00000000e+00,  6.12000000e-17,  0.00000000e+00,
        1.00000000e+00,  0.00000000e+00,  1.00000000e+00,  9.56234827e-01,
        2.92600336e-01])

In [17]:
time_steps=2
num_features=21

In [18]:
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=0)

In [19]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02]])

In [20]:
train_y[0]

array([], dtype=float64)

In [21]:
time_steps =24
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=1,target_len=2)

In [22]:
train_X[0]

array([[ 1.69100212e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         2.58819045e-01,  9.65925826e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.41395233e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e-01,
         5.00000000e-01,  8.66025404e-01,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         6.12323400e-17, -9.99768502e-01,  2.15160974e-02],
       [ 1.26232372e-01,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00, -8.66025404e-01,
         5.00000000e-01, -8.66025404e-01, -5.00000000e

In [23]:
train_y[0]

array([1., 1.])

In [24]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 4.945459365844727 sec


In [26]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                    verbose = verbose)

Epoch 1/10
2649/2653 [============================>.] - ETA: 0s - loss: 0.0174 - mae: 0.0174 - mape: 395.5573
Epoch 00001: val_loss improved from 0.01671 to 0.01656, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E1-cp-0001-loss0.02.h5
2653/2653 [==============================] - 28s 11ms/step - loss: 0.0174 - mae: 0.0174 - mape: 395.0338 - val_loss: 0.0166 - val_mae: 0.0166 - val_mape: 7.2706
Epoch 2/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0169 - mae: 0.0169 - mape: 487.0553
Epoch 00002: val_loss improved from 0.01656 to 0.01570, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E1-cp-0002-loss0.02.h5
2653/2653 [==============================] - 30s 11ms/step - loss: 0.0169 - mae: 0.0169 - mape: 486.7708 - val_loss: 0.0157 - val_mae: 0.0157 - val_mape: 7.0236
Epoch 3/10
2650/2653 [============================>.] - ETA: 0s - loss: 0.0161 - mae: 0.0161 - mape: 341.2231
Epoch 00003: val_loss did not impr

In [27]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E1-cp-0010-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})


y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 190.28
Median Absolute Error (MedAE): 157.01
Mean Squared Error (MSE): 59040.62
Root Mean Squared Error (RMSE): 242.98
Mean Absolute Percentage Error (MAPE): 1.29 %
Median Absolute Percentage Error (MDAPE): 1.09 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# Fine Tuning

In [28]:
checkpoints = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E1-cp-0010-loss0.01.h5'
start_epoch= 20

In [29]:
import os
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, InputLayer
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras.backend as K

# Define your model builder function (replace PC.build)
def build_model(time_steps, num_features, reg=0.0005):
    model = Sequential()
    model.add(InputLayer(input_shape=(time_steps, num_features)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(1))  # Adjust output layer size according to your task
    return model


# Path to save/load your model checkpoint
model_path = r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'

# Construct the callback to save only the *best* model based on validation loss
EpochCheckpoint1 = ModelCheckpoint(
    model_path,
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1]

# Variable 'model' for holding your model object
model = None

# Check if checkpoint file exists
if not os.path.exists(model_path):
    print("[INFO] No checkpoint found. Compiling new model...")
    model = build_model(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(learning_rate=1e-3)
    model.compile(loss="mae", optimizer=opt, metrics=["mae", "mape"])
else:
    print(f"[INFO] Loading model from {model_path} ...")
    model = load_model(model_path)

    # Update the learning rate
    print(f"[INFO] Old learning rate: {K.get_value(model.optimizer.lr)}")
    K.set_value(model.optimizer.lr, 1e-4)
    print(f"[INFO] New learning rate: {K.get_value(model.optimizer.lr)}")

# Now you can train your model with callbacks
# Example:
# model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=..., callbacks=callbacks)


[INFO] No checkpoint found. Compiling new model...


In [30]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0252 - mae: 0.0252 - mape: 172.4312
Epoch 00001: val_loss improved from inf to 0.01767, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E2-cp-0001-loss0.02.h5
2653/2653 [==============================] - 190s 72ms/step - loss: 0.0252 - mae: 0.0252 - mape: 172.4312 - val_loss: 0.0177 - val_mae: 0.0177 - val_mape: 7.2598
Epoch 2/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0134 - mae: 0.0134 - mape: 79.6321
Epoch 00002: val_loss improved from 0.01767 to 0.01063, saving model to E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 200s 76ms/step - loss: 0.0134 - mae: 0.0134 - mape: 79.6321 - val_loss: 0.0106 - val_mae: 0.0106 - val_mape: 4.3480
Epoch 3/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0107 - mae: 0.0107 - mape: 18.6516
Epoch 00003: val_loss did not improve f

In [ ]:

model = load_model(r'E:\ELECTRICAL ENGINEERIGH(COMNICATN)\Semester-8th\ML Lab\lab7\E2-cp-0007-loss0.01.h5',
                   custom_objects={'mae': mean_absolute_error})
#model = load_model(r'C:\Users\PMLS\shahid (ML-lab)\lab7\E1-cp-0016-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

Mean Absolute Error (MAE): 113.26
Median Absolute Error (MedAE): 88.89
Mean Squared Error (MSE): 22267.31
Root Mean Squared Error (RMSE): 149.22
Mean Absolute Percentage Error (MAPE): 0.78 %
Median Absolute Percentage Error (MDAPE): 0.61 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
